In [177]:
import pandas as pd
import json
import os
import numpy as np
import datetime
import time
import requests
from bs4 import BeautifulSoup
from PIL import Image
import io
pd.set_option('display.max_columns', None)

In [ ]:
class FootballDataScraper:

    def __init__(self, config_file):
        
        self.config_file = config_file
        self.headers = {'User-Agent': 'Mozilla/5.0'}
        self.championnats = self.load_configs()
        self.data_folder = "Database"
        self.logos_folder = "Logos"
        self.futur_data_folder = "Futur_data"
        self.last_request_time = None
        self.request_interval = 3  # secondes
        self.max_seasons = 6

    def load_configs(self):
        # loading config of championship containing which championship are used and their urls
        return pd.read_csv(self.config_file)

    def scrape_or_update(self):
        all_data = {}
        for _, config in self.championnats.iterrows():
            championnat = config['Championnat']
            url = config['URL']
            csv_file_path = os.path.join(self.data_folder, f"{championnat}_data.csv")

            if os.path.exists(csv_file_path):
                data = pd.read_csv(csv_file_path)
                last_update = pd.to_datetime((data['Date'] + ' ' + data['Time']).max())

                if datetime.datetime.now() > last_update + datetime.timedelta(hours = 2):
                    updated_data = self.update_data(url, data)
                    data_updated = updated_data[0].copy()
                    all_data[championnat] = data_updated[data_updated["Comp"]==championnat]
                    futur_data = self.save_futur_data(updated_data[1], championnat)
                    self.save_data(futur_data, os.path.join(self.futur_data_folder, f"{championnat}_futur_data.csv"))
                    
            else:
                data = self.scrape_data(url)
                all_data[championnat] = data[data["Comp"]==championnat]

            self.save_data(all_data[championnat], csv_file_path)
        return all_data

    def scrape_data(self, url):
        
        all_seasons_data = []

        for saison in range(self.max_seasons):

            self.rate_limit()

            response = requests.get(url, headers=self.headers)
            soup = BeautifulSoup(response.text, 'html.parser')
            
            teams_urls = ["https://fbref.com" + equipe.get("href") 
                        for equipe in soup.select("table.stats_table")[0].find_all("a") 
                        if "squads" in equipe.get("href", "")]

            url = f"https://fbref.com{soup.find('a', class_='button2 prev').get('href')}"

            for team_url in teams_urls:

                self.rate_limit()
                team_response = requests.get(team_url, headers=self.headers)
                try:
                    team_data = pd.read_html(team_response.text, match="Scores")[0]
                except ValueError as e:
                    # Gestion de l'exception si aucune table n'est trouvée
                    print(f"Aucune table trouvée dans {team_url} - Erreur: {e}")
                    continue  # Passe à l'itération suivante de la boucle

                team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
                team_data["Team"] = team_name

                self.scrape_and_save_logo(team_url, team_name)

                # Récupération des URLs pour les statistiques détaillées
                url_stats = {
                    f"https://fbref.com{a.get('href')}" 
                    for a in BeautifulSoup(team_response.text, 'html.parser').find_all("a") 
                    if "matchlogs/all_comps" in a.get('href', '') and 
                    any(substring in a.get('href', '') for substring in ["passing/", "shooting", "possession/", "defense/", "keeper"])
                }

                # Traitement des statistiques détaillées
                for stats_url in url_stats:

                    self.rate_limit()

                    stats_response = requests.get(stats_url, headers=self.headers)

                    try:
                        detailed_stats = pd.read_html(stats_response.text)[0]
                    except ValueError as e:
                        print(f"Aucune table trouvée dans {stats_url} - Erreur: {e}")
                        continue  # Passe à l'itération suivante de la boucle

                    # Nettoyage des colonnes du DataFrame
                    if detailed_stats.columns.nlevels > 1:
                        detailed_stats.columns = [f"{col}_{branch}" 
                                                if "For" not in col and "Unnamed:" not in col 
                                                else f"{branch}" 
                                                for col, branch in detailed_stats.columns]

                    columns_to_drop = ["Time", "Comp", "Round", "Day", "Venue", "Result", "GF", "GA", "Opponent", "Poss"] + [col for col in detailed_stats.columns if 'Report' in col]
                    columns_to_drop = [col for col in columns_to_drop if col in detailed_stats.columns]

                    detailed_stats.drop(columns_to_drop, axis=1, inplace=True)

                    team_data = team_data.merge(detailed_stats, on="Date")

                # Ajout des données de l'équipe au résultat global
                all_seasons_data.append(team_data)

        return pd.concat(all_seasons_data, ignore_index=True)

    def update_data(self, url, data):

        futur_data = []
        all_data = []
        response = requests.get(url, headers=self.headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        teams_urls = ["https://fbref.com" + equipe.get("href") 
                    for equipe in soup.select("table.stats_table")[0].find_all("a") 
                    if "squads" in equipe.get("href", "")]

        # Traitement similaire à scrape_ligue1_data() pour chaque équipe
        for team_url in teams_urls:

            self.rate_limit()

            team_response = requests.get(team_url, headers=self.headers)
            try:
                team_data = pd.read_html(team_response.text, match="Scores")[0]
            except ValueError as e:
                # Gestion de l'exception si aucune table n'est trouvée
                print(f"Aucune table trouvée dans {team_url} - Erreur: {e}")
                continue  # Passe à l'itération suivante de la boucle

            team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
            team_data["Team"] = team_name

            futur_data.append(team_data)


            # Récupération des URLs pour les statistiques détaillées
            url_stats = {
                f"https://fbref.com{a.get('href')}" 
                for a in BeautifulSoup(team_response.text, 'html.parser').find_all("a") 
                if "matchlogs/all_comps" in a.get('href', '') and 
                any(substring in a.get('href', '') for substring in ["passing/", "shooting", "possession/", "defense/", "keeper"])
            }

            for stats_url in url_stats:

                self.rate_limit()

                stats_response = requests.get(stats_url, headers=self.headers)
                
                try:
                    detailed_stats = pd.read_html(stats_response.text)[0]
                except ValueError as e:
                    print(f"Aucune table trouvée dans {stats_url} - Erreur: {e}")
                    continue  # Passe à l'itération suivante de la boucle


                # Nettoyage des colonnes du DataFrame
                if detailed_stats.columns.nlevels > 1:
                    detailed_stats.columns = [f"{col}_{branch}" 
                                            if "For" not in col and "Unnamed:" not in col 
                                            else f"{branch}" 
                                            for col, branch in detailed_stats.columns]

                columns_to_drop = ["Time", "Comp", "Round", "Day", "Venue", "Result", "GF", "GA", "Opponent", "Poss"] + [col for col in detailed_stats.columns if 'Report' in col]
                columns_to_drop = [col for col in columns_to_drop if col in detailed_stats.columns]

                detailed_stats.drop(columns_to_drop, axis=1, inplace=True)

                team_data = team_data.merge(detailed_stats, on="Date")

            # Ajout des données de l'équipe au résultat global
            all_data.append(team_data)

        new_data = pd.concat(all_data, ignore_index=True)

        # Concaténation de la base initiale et de la base nouvelle
        concatenated_df = pd.concat([data, new_data])
        
        # Suppression des doublons basée sur les colonnes "Date", "Team" et "Opponent"
        concatenated_df = concatenated_df.drop_duplicates(subset=['Date', 'Team', 'Opponent'])
            
        return concatenated_df.reset_index(drop=True), pd.concat(futur_data, ignore_index=True)

    def save_data(self, data, file_path):
        # Enregistrer les données dans un fichier CSV
        data.to_csv(file_path, index=False)

    def rate_limit(self):
        if self.last_request_time is not None:
            elapsed_time = time.time() - self.last_request_time
            if elapsed_time < self.request_interval:
                time.sleep(self.request_interval - elapsed_time)
        self.last_request_time = time.time()

    def access_data(self, championnat):
        csv_file_path = os.path.join(self.data_folder, f"{championnat}_data.csv")
        
        if os.path.exists(csv_file_path):
            data = pd.read_csv(csv_file_path)
            return data
        else:
            print(f"Les données pour le championnat '{championnat}' n'existent pas.")

    def scrape_and_save_logo(self, page_url, file_name):

        file_path = os.path.join(self.logos_folder, file_name + '.png')
        if os.path.exists(file_path):
            print(f"Le logo existe déjà à {file_path}")
            return

        try:
            self.rate_limit()
            response = requests.get(page_url, headers=self.headers)
            soup = BeautifulSoup(response.text, 'html.parser')
            logo_img = soup.find('img', {'class': 'teamlogo'})

            if logo_img and logo_img.get('src'):
                logo_url = logo_img['src']
                self.rate_limit()
                img_response = requests.get(logo_url)

                if img_response.status_code == 200:
                    # Convertir l'image en PNG
                    image = Image.open(io.BytesIO(img_response.content))
                    file_path = os.path.join(self.logos_folder, file_name)
                    if not file_path.endswith('.png'):
                        file_path += '.png'
                    image.save(file_path, 'PNG')
                    print(f"Logo sauvegardé en PNG à {file_path}")
                else:
                    print("Erreur lors du téléchargement de l'image")
            else:
                print("Logo non trouvé sur la page")

        except Exception as e:
            print(f"Une erreur s'est produite : {e}")

    def save_futur_data(self, data, championnat):

        """
        À partir du scrapping on récupère un DataFrame qui contient les futurs journées, il faut le mettre en forme.
        args: DataFrame, le mapping ligue1
        """

        # Supprimer les lignes où les colonnes 'Date', 'Time' et 'Round' sont manquantes.
        data.dropna(subset=["Date", "Time", "Round"], inplace=True)
        data['DateTime'] = pd.to_datetime((data['Date'] + ' ' + data['Time']))
        data = data[data["Comp"] == championnat]

        data = data[data['DateTime'] >= datetime.datetime.now()].sort_values(by="DateTime")


        if data.empty != True:
                first_date = data['DateTime'].iloc[0]
        else:
            return None
        
        ten_days = datetime.timedelta(days=10) + first_date

        # Filtrer pour garder seulement les matchs programmés dans les 10 jours suivant la 'premiere_date_proche'.
        data = data[data['DateTime'] <= ten_days]

        if data.empty == True:
            return None

        else:# Retourner le DataFrame s'il n'est pas vide, sinon retourner None.
            csv_file_path = os.path.join(self.futur_data_folder, f"{championnat}_futur_data.csv")
            if os.path.exists(csv_file_path):
                futur_data = pd.read_csv(csv_file_path)
                last_update = pd.to_datetime((futur_data['Date'] + ' ' + futur_data['Time'])).max()
                if last_update == pd.to_datetime((data['Date'] + ' ' + data['Time'])).max():
                    return futur_data
                else:
                    return pd.concat([futur_data, data]).drop_duplicates(subset=["Team", "Opponent", "Date"])
            else:
                return data


    


In [142]:
allo = FootballDataScraper("config.csv")
allo.scrape_or_update()

/tmp/ipykernel_312670/1591475410.py:145: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  team_data = pd.read_html(team_response.text, match="Scores")[0]
/tmp/ipykernel_312670/1591475410.py:172: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  detailed_stats = pd.read_html(stats_response.text)[0]
/tmp/ipykernel_312670/1591475410.py:172: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  detailed_stats = pd.read_html(stats_response.text)[0]
/tmp/ipykernel_312670/1591475410.py:172: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a '

Aucune table trouvée dans https://fbref.com/en/squads/19538871/Manchester-United-Stats - Erreur: No tables found matching pattern 'Scores'


/tmp/ipykernel_312670/1591475410.py:172: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  detailed_stats = pd.read_html(stats_response.text)[0]
/tmp/ipykernel_312670/1591475410.py:172: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  detailed_stats = pd.read_html(stats_response.text)[0]
/tmp/ipykernel_312670/1591475410.py:172: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  detailed_stats = pd.read_html(stats_response.text)[0]
/tmp/ipykernel_312670/1591475410.py:172: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' 

{'La Liga':             Date   Time     Comp         Round  Day Venue Result   GF   GA  \
 0     2023-08-12  17:00  La Liga   Matchweek 1  Sat  Away      D  1.0  1.0   
 1     2023-08-20  19:00  La Liga   Matchweek 2  Sun  Home      W  3.0  0.0   
 2     2023-08-26  21:30  La Liga   Matchweek 3  Sat  Away      W  2.0  1.0   
 3     2023-09-03  14:00  La Liga   Matchweek 4  Sun  Home      W  1.0  0.0   
 4     2023-09-18  21:00  La Liga   Matchweek 5  Mon  Away      W  4.0  2.0   
 ...          ...    ...      ...           ...  ...   ...    ...  ...  ...   
 4292  2024-01-28  21:00  La Liga  Matchweek 22  Sun  Away      L  0.0  2.0   
 4309  2024-01-28  18:30  La Liga  Matchweek 22  Sun  Away      D  1.0  1.0   
 4340  2024-01-28  18:30  La Liga  Matchweek 22  Sun  Home      D    1    1   
 4346  2024-01-28  14:00  La Liga  Matchweek 22  Sun  Home      L  0.0  1.0   
 4348  2024-01-28  16:15  La Liga  Matchweek 22  Sun  Home      D    0    0   
 
              Opponent   xG  xGA  Poss 

In [179]:
class FootballMain:

    def __init__(self):

        self.data_folder = "Database"
        self.logos_folder = "Logos"
        self.futur_data_folder = "Futur_data"
        
    def data(self, league):
        data = self._raw_data(league)
        return ProcessingFootball().initial_processing(data)

    def data_with_features(self, league):
        data = self._raw_data(league) 
        return ProcessingFootball().features_processing(data)

    def data_for_prediction(self, league):
        data = self._raw_data(league) 
        return ProcessingFootball().prediction_processing(data)

    def data_for_futur_prediction(self, league):
        data = self._raw_data(league)
        futur_data = self._raw_futur_data(league)
        return ProcessingFootball().futur_prediciton_processing(data, futur_data)
        
    def _raw_data(self, league):
        path = os.path.join(self.data_folder, f"{league}_data.csv")
        data = pd.read_csv(path)
        data = self._mapped_data(data, league)
        return data

    def _raw_futur_data(self, league):
        path = os.path.join(self.futur_data_folder, f"{league}_futur_data.csv")
        data = pd.read_csv(path)
        data = self._mapped_data(data, league)
        return data

    def _mapped_data(self, data, league):
        with open('mapping_team.json', 'r', encoding='utf-8') as file:
            mappings = json.load(file)
        mapping = mappings.get(league, {})
        data['Opponent'] = data['Opponent'].map(mapping).fillna(data['Opponent'])
        data['Team'] = data['Team'].map(mapping).fillna(data['Team'])
        return data
    
    """
    def _read_raw_all_data(self):

        list_data = []
        for fichier in os.listdir(self.data_folder):
            if fichier.endswith('.csv'):
                path = os.path.join(self.data_folder, fichier)
                data = _read_raw_data()
                list_data.append(data)
        
        return pd.concat(list_data, ignore_index=True)
    """









In [230]:
import pandas as pd

def minmax_scale(x):
    return (x - x.min()) / (x.max() - x.min())

# Ensuite, utilisez cette fonction avec .transform() pour appliquer la normalisation à chaque groupe



class ProcessingFootball:

    def __init__(self):

        self.list_columns = [
            "Total Shots", "Shots on Target", "Shots on Target %", "Goals per Shot", "Total Touches", 
            "Touches in Defensive Penalty Area", "Touches in Defensive Third", "Touches in Midfield Third", 
            "Touches in Attacking Third", "Touches in Attacking Penalty Area", "Dribbles Attempted", 
            "Successful Dribbles", "Successful Dribble %", "Total Carries", "Total Carry Distance", 
            "Progressive Carry Distance", "Progressive Carries", "Carries into Final Third", 
            "Carries into Penalty Area", "Tackles", "Tackles Won", "Tackles in Defensive Third", 
            "Tackles in Midfield Third", "Tackles in Attacking Third", "Dribblers Tackled", 
            "Total Dribbles Against", "Defensive Dribblers Win %", "Interceptions", "Errors Leading to Goal", 
            "Key Passes", "Passes Completed", "Passes Attempted", "Passes into Final Third", 
            "Progressive Passes", "Shots on Target Against", "Keeper Saves", "Keeper Save Percentage"]
        self.foundations_columns = ["DateTime", "Comp", "Season", "Round", "Day", "Venue", "Result", "GF", "GA", "Opponent", "xG", "xGA", "Poss", "Attendance", "Captain", "Formation", "Referee", "Match Report", "Notes", "Team", "Minus 1.5 Goals", "Minus 2.5 Goals", "Minus 3.5 Goals"]


    def initial_processing(self, data): 
        data = self._prepare_basic_columns(data)
        data = self._rename_and_drop_columns(data)
        data = self._calculate_cumulatives_features(data)
        data = self._calculate_ranking(data)
        data = self._features_bookmaker_creation(data)
        return data

    def features_processing(self, data):
        data = self.initial_processing(data)
        data = self.calculate_features_for_model(data)
        return data


    def prediction_processing(self, data):
        data = self.features_processing(data)
        data = self._keep_columns_for_model(data)
        data = self._merge_2_rows_in_one(data)
        return data

    def futur_prediciton_processing(self,data, data_next_match):
        data = self.initial_processing(data)
        data_next_match = self._prepare_basic_columns(data_next_match)
        glob_data = pd.concat([data, data_next_match], sort=False).reset_index(drop=True)
        glob_data = self.calculate_features_for_model(glob_data)
        #glob_data.dropna(subset=['Total Shots_5_Last_Matches_Average'], inplace=True)
        glob_data = self._keep_columns_for_model(glob_data)
        glob_data = self._merge_2_rows_in_one(glob_data)
        return glob_data

    def calculate_features_for_model(self, data):
        data = self._calculate_lagged_features(data)
        data = self._calculate_5_last_match_average(data, self.list_columns)
        data = self._calculate_5_last_match_sum(data, self.list_columns)
        data = self._calculate_5_last_match_std(data, self.list_columns)
        data = self._calculate_5_last_match_form(data)
        return data

    def _prepare_basic_columns(self, data):

        if 'Date' in data.columns and 'Time' in data.columns:
            data['DateTime'] = pd.to_datetime(data['Date'] + ' ' + data['Time'])
            data.drop(["Date", "Time"], axis=1, inplace=True)
            data = data[['DateTime'] + [col for col in data.columns if col != 'DateTime']]
            data['Season'] = data['DateTime'].apply(
                lambda x: f"{x.year}-{x.year + 1}" if x.month >= 8 else f"{x.year - 1}-{x.year}")


        if 'Round' in data.columns:
            data = data[data['Round'].str.startswith('Matchweek')]
            if 'Round' in data.columns:  # Vérification à nouveau en cas de changement
                data['Round'] = data['Round'].str.extract(r'(\d+)').astype(int)


        if 'Formation' in data.columns:
            data['Formation'] = data['Formation'].apply(
                lambda x: x.replace('◆', '') if pd.notnull(x) else x)

        if 'GF' in data.columns and 'GA' in data.columns:
            if data[['GF', 'GA']].notnull().all().all():
                data[['GF', 'GA']] = data[['GF', 'GA']].astype(float).astype(int)
            data['GD'] = data['GF'] - data['GA']
            data["Total_Goals"] = data["GF"] + data["GA"]

        if 'Result' in data.columns:
            data['Points'] = data.apply(lambda row: {'W': 3, 'D': 1, 'L': 0}.get(row['Result']) if pd.notnull(row['Result']) else np.nan, axis=1)
            
        return data


    def _calculate_cumulatives_features(self, data):

        """ Creation of cumulatives columns per season (examle : Points cumulated at each date, goals for cumulated...)
        """

        data.sort_values(by=['Season', 'Round', 'Team'], inplace=True)
        data.reset_index(drop=True, inplace=True)

        cumulative_cols = data.groupby(['Season', 'Team']).agg({
            'Points': 'cumsum',
            'GD': 'cumsum',
            'GF': 'cumsum',
            'GA': 'cumsum'
        }).reset_index()

        data[['Points_Cum', 'GD_Cum', 'GF_Cum', 'GA_Cum']] = cumulative_cols[['Points', 'GD', 'GF', 'GA']]
        
        return data


    def _calculate_ranking(self, data):

        data.sort_values(by=['Season', 'Round', 'Points_Cum', 'GD_Cum'], ascending=[True, True, False, False], inplace=True)
        data['Ranking'] = data.groupby(['Season', 'Round']).cumcount() + 1

        return data

    def _features_bookmaker_creation(self, data):

        data["Minus 1.5 Goals"] = (data["Total_Goals"] <= 1.5).astype(int)
        data["Minus 2.5 Goals"] = (data["Total_Goals"] <= 2.5).astype(int)
        data["Minus 3.5 Goals"] = (data["Total_Goals"] <= 3.5).astype(int)

        return data

    def find_mapping(self, championnat):
        
        # show the difference of team name between the two columns
        # help to modify the file for mapping

        data = read_data(championnat)
        problem_names_1 = set(data['Team'].unique()) - set(data['Opponent'].unique())
        problem_names_2 = set(data['Opponent'].unique()) - set(data['Team'].unique())

        print(sorted(problem_names_1))
        print(sorted(problem_names_2))


    def _rename_and_drop_columns(self, data):
    
        with open('mapping_columns.json', 'r', encoding='utf-8') as file:
            mappings = json.load(file)
        
        precise_renaming_dict = mappings.get("Columns", {})
        
        for old_col, new_col in precise_renaming_dict.items():
            if old_col in data.columns:
                data.rename(columns={old_col: new_col}, inplace=True)

        columns_to_drop = [
            "Standard_Gls", "Launched_Cmp", "Launched_Att", "Launched_Cmp%", 
            "Passes_Att (GK)", "Passes_Thr", "Passes_Launch%", "Passes_AvgLen", 
            "Goal Kicks_Att", "Goal Kicks_Launch%", "Goal Kicks_AvgLen", 
            "Crosses_Opp", "Crosses_Stp", "Crosses_Stp%", "Sweeper_#OPA", 
            "Sweeper_AvgDist", "Penalty Kicks_PKatt", "Performance_GA", 
            "Performance_PSxG", "Performance_PSxG+/-", "Receiving_Rec", 
            "Touches_Live", "Standard_PKatt", "Receiving_PrgR"
        ]

        columns_to_drop = [col for col in columns_to_drop if col in data.columns]
        data.drop(columns=columns_to_drop, inplace=True)

        return data

    def _calculate_lagged_features(self, data):

        data.sort_values(by=['Season', 'Round', 'Team'], inplace=True)
        data.reset_index(drop=True, inplace=True)

        lag_cols = ['Points_Cum', 'GD_Cum', 'GF_Cum', 'GA_Cum']
        data[[f'{col}_Lag' for col in lag_cols]] = data.groupby(['Season', 'Team'])[lag_cols].shift(1)

        # Décalage du classement pour chaque équipe
        data['Ranking_Lag'] = data.groupby(['Team'])['Ranking'].shift(1)

        return data

    def _calculate_5_last_match_form(self, data):

        data.sort_values(by=['Season', 'Round', 'Points_Cum', 'GD_Cum'], ascending=[True, True, False, False], inplace=True)
        
        data["5_Last_Matches_Win"] = data.groupby(['Season', 'Team'])['Result'].transform(
                                    lambda x: (x == "W").shift(1).rolling(window=5, min_periods=1).sum().fillna(0))

        data["5_Last_Matches_Loose"] = data.groupby(['Season', 'Team'])['Result'].transform(
                                    lambda x: (x == "L").shift(1).rolling(window=5, min_periods=1).sum().fillna(0))

        return data
    
    def _calculate_5_last_match_average(self, data, list_columns):

        #data.sort_values(by=['Season', 'Round', 'Points_Cum', 'GD_Cum'], ascending=[True, True, False, False], inplace=True)

        new_columns = pd.DataFrame(index=data.index)

        for col in list_columns:
            new_columns[f'{col}_5_Last_Matches_Average'] = data.groupby(['Season', 'Team'])[col].transform(lambda x: x.shift(1).rolling(window=5, min_periods=5).mean())

        data = pd.concat([data, new_columns], axis=1)

        return data

    def _calculate_5_last_match_sum(self, data, list_columns):

        data.sort_values(by=['Season', 'Round', 'Points_Cum', 'GD_Cum'], ascending=[True, True, False, False], inplace=True)

        new_columns = pd.DataFrame(index=data.index)

        for col in list_columns:
            new_columns[f'{col}_5_Last_Matches_Sum'] = data.groupby(['Season', 'Team'])[col].transform(lambda x: x.shift(1).rolling(window=5, min_periods=5).sum())

        data = pd.concat([data, new_columns], axis=1)

        return data

    def _calculate_5_last_match_std(self, data, list_columns):

        data.sort_values(by=['Season', 'Round', 'Points_Cum', 'GD_Cum'], ascending=[True, True, False, False], inplace=True)

        new_columns = pd.DataFrame(index=data.index)

        for col in list_columns:
            new_columns[f'{col}_5_Last_Matches_Std'] = data.groupby(['Season', 'Team'])[col].transform(lambda x: x.shift(1).rolling(window=5, min_periods=5).std())

        data = pd.concat([data, new_columns], axis=1)

        return data
    

    def _calculate_scaled_season_average_mean(self, data, list_columns):

        data.sort_values(by=['Season', 'Round', 'Points_Cum', 'GD_Cum'], ascending=[True, True, False, False], inplace=True)

        new_columns = pd.DataFrame(index=data.index)

        for col in list_columns:
            new_columns[f'{col}_5_Last_Matches_Std'] = data.groupby(['Season', 'Team'])[col].transform(lambda x: x.shift(1).rolling(window=5, min_periods=5).std())

        data = pd.concat([data, new_columns], axis=1)
        RRR = A.groupby(['Season', 'Round'])['Tackles'].transform(minmax_scale)
        return data




    def _keep_columns_for_model(self, data):

        columns = self.foundations_columns
        cols_to_keep = [col for col in data.columns if "_Lag" in col or "_5_Last_Matches" in col or col in columns]

        data = data[cols_to_keep]

        return data

    def _merge_2_rows_in_one(self, data):

        data['MatchID'] = data['DateTime'].astype(str) + '-' + data[['Team', 'Opponent']].apply(sorted, axis=1).str.join('-vs-')

        fixed_columns = ['DateTime', 'Comp', 'Round', 'Day', 'MatchID', 'Season', 'Attendance', 'Referee', 'Match Report', 'Notes', "Minus 1.5 Goals", "Minus 2.5 Goals", "Minus 3.5 Goals"]
        
        moving_variables = [col for col in data.columns if col not in fixed_columns]

        data_home = data[data['Venue'] == 'Home'].copy()
        data_away = data[data['Venue'] == 'Away'].copy()

        rename_dict_home = {col: f"{col}_Home" for col in moving_variables}
        rename_dict_away = {col: f"{col}_Away" for col in moving_variables}
        data_home.rename(columns=rename_dict_home, inplace=True)
        data_away.rename(columns=rename_dict_away, inplace=True)

        data = pd.merge(data_home, data_away, on=fixed_columns, how='inner')

        conditions = [
            data['Result_Home'] == 'W',
            data['Result_Away'] == 'W',
            data['Result_Away'] == 'D'
        ]
        choices = ['W_Home', 'W_Away', 'D']
        data['Result'] = np.select(conditions, choices, default=np.nan)

        data.rename(columns={"Team_Home": "Team Home", "Team_Away": "Team Away"}, inplace=True)

        return data



#Keeper Save Percentage_5_Last_Matches_Std     nan = 432 donc les enlever ou les fixer à zéro
#minmax scaler
#regarder les groupby index (comprendre)
#creer les average de donnees en saison
    



In [212]:
test.data('Ligue 1')

/tmp/ipykernel_476548/2884795212.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Season'] = data['DateTime'].apply(


,DateTime,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes,Team,Tackles,Tackles Won,Tackles in Defensive Third,Tackles in Midfield Third,Tackles in Attacking Third,Dribblers Tackled,Total Dribbles Against,Defensive Dribblers Win %,Defensive Challenges Lost,Total Blocks,Shot Blocks,Pass Blocks,Interceptions,Tackles Plus Interceptions,Clearances,Errors Leading to Goal,Total Touches,Touches in Defensive Penalty Area,Touches in Defensive Third,Touches in Midfield Third,Touches in Attacking Third,Touches in Attacking Penalty Area,Dribbles Attempted,Successful Dribbles,Successful Dribble %,Dribbles Tackled,Dribble Tackle %,Total Carries,Total Carry Distance,Progressive Carry Distance,Progressive Carries,Carries into Final Third,Carries into Penalty Area,Carries Miscontrolled,Carries Dispossessed,Total Shots,Shots on Target,Shots on Target %,Goals per Shot,Goals per Shot on Target,Average Shot Distance,Free Kicks Taken,Penalty Kicks Scored,Expected Goals,Non-Penalty Expected Goals,Non-Penalty Expected Goals per Shot,Goal Difference vs Expected Goals,Non-Penalty Goal Difference vs Expected Goals,Passes Completed,Passes Attempted,Pass Completion %,Total Pass Distance,Progressive Pass Distance,Short Passes Completed,Short Passes Attempted,Short Pass Completion %,Medium Passes Completed,Medium Passes Attempted,Medium Pass Completion %,Long Passes Completed,Long Passes Attempted,Long Pass Completion %,Assists,Expected Assists Goals,Expected Assists,Key Passes,Passes into Final Third,Passes into Penalty Area,Crosses into Penalty Area,Progressive Passes,Shots on Target Against,Keeper Saves,Keeper Save Percentage,Clean Sheets,Penalty Kicks Against,Penalty Kicks Against Saved,Penalty Kicks Against Missed,Season,GD,Total_Goals,Points,Points_Cum,GD_Cum,GF_Cum,GA_Cum,Ranking,Minus 1.5 Goals,Minus 2.5 Goals,Minus 3.5 Goals
8,2018-08-10 20:45:00,Ligue 1,1,Fri,Home,W,4,0,Toulouse,3.2,0.3,60.0,60756.0,Dimitri Payet,4-2-3-1,Ruddy Buquet,Match Report,NaN,Marseille,29.0,20.0,11.0,11.0,7.0,15.0,26.0,57.7,11.0,8.0,0.0,8.0,7.0,36,14.0,0.0,660.0,45.0,205.0,313.0,153.0,30.0,13.0,6.0,46.2,7.0,53.8,438.0,2287.0,1333.0,31.0,26.0,8.0,15.0,12.0,22.0,9.0,40.9,0.14,0.33,18.6,2.0,1,3.2,2.5,0.11,0.8,0.5,448.0,542.0,82.7,7324.0,2540.0,210.0,232.0,90.5,147.0,167.0,88.0,54.0,87.0,62.1,2.0,1.7,0.9,14.0,18.0,6.0,3.0,29.0,1.0,1.0,100.0,1.0,0,0.0,0.0,2018-2019,4,4,3,3,4,4,0,1,0,0,0
14,2018-08-12 21:00:00,Ligue 1,1,Sun,Home,W,3,0,Caen,3.0,0.2,65.0,47289.0,Thiago Silva,4-3-3,Benoît Bastien,Match Report,NaN,Paris Saint Germain,16.0,7.0,5.0,8.0,3.0,5.0,9.0,55.6,4.0,7.0,2.0,5.0,7.0,23,13.0,0.0,920.0,29.0,131.0,584.0,209.0,25.0,25.0,14.0,56.0,11.0,44.0,611.0,2939.0,1482.0,22.0,18.0,8.0,18.0,13.0,11.0,8.0,72.7,0.27,0.38,15.2,2.0,0,3.0,3.0,0.28,0.0,0.0,742.0,819.0,90.6,11608.0,3214.0,413.0,444.0,93.0,245.0,266.0,92.1,54.0,66.0,81.8,2.0,1.8,2.2,6.0,62.0,10.0,1.0,63.0,1.0,1.0,100.0,1.0,0,0.0,0.0,2018-2019,3,3,3,3,3,3,0,2,0,0,1
6,2018-08-11 20:00:00,Ligue 1,1,Sat,Home,W,3,1,Rennes,1.7,0.5,56.0,25708.0,Adama Soumaoro,4-2-3-1,Willy Delajod,Match Report,NaN,Lille,15.0,10.0,7.0,8.0,0.0,5.0,10.0,50.0,5.0,8.0,5.0,3.0,18.0,33,19.0,1.0,629.0,49.0,193.0,304.0,135.0,30.0,13.0,10.0,76.9,3.0,23.1,344.0,1850.0,997.0,21.0,16.0,6.0,14.0,10.0,15.0,6.0,40.0,0.20,0.50,19.2,1.0,0,1.7,1.7,0.11,1.3,1.3,427.0,524.0,81.5,6906.0,2647.0,187.0,209.0,89.5,161.0,194.0,83.0,40.0,64.0,62.5,3.0,1.6,1.1,11.0,26.0,12.0,3.0,37.0,2.0,1.0,50.0,0.0,0,0.0,0.0,2018-2019,2,4,3,3,2,3,1,3,0,0,0
7,2018-08-12 15:00:00,Ligue 1,1,Sun,Home,W,2,0,Amiens,1.4,0.4,67.0,48263.0,Marcelo,4-3-1-2,Franck Schneider,Match Report,NaN,Lyon,13.0,7.0,4.0,9.0,0.0,5.0,13.0,38.5,8.0,12.0,5.0,7.0,11.0,24,14.0,0.0,712.0,32.0,156.0,401.0,164.0,21.0,28.0,16.0,57.1,12.0,42.9,446.0,2887.0,1638.0,22.0,23.0,9.0,12.0,11.0,15.0,7.0,46.7,0.13,0.29,18.3,4.0,0,1.4,1.4,0.09,0.6,0.6,523.0,608.0,86.0,9029.0,2560.0,229.0,248.0,92.3,231.0,251.0,92.0,48.0,76.0,63.2,1.0,1.2,1.0,10.0,52.0,2.0,0.0,53.0,1.0,1

In [228]:
A = FootballMain().data_with_features("Ligue 1")

/tmp/ipykernel_476548/2884795212.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Season'] = data['DateTime'].apply(


In [227]:
A[A['Team']=='Lyon'].head(10)

,DateTime,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes,Team,Tackles,Tackles Won,Tackles in Defensive Third,Tackles in Midfield Third,Tackles in Attacking Third,Dribblers Tackled,Total Dribbles Against,Defensive Dribblers Win %,Defensive Challenges Lost,Total Blocks,Shot Blocks,Pass Blocks,Interceptions,Tackles Plus Interceptions,Clearances,Errors Leading to Goal,Total Touches,Touches in Defensive Penalty Area,Touches in Defensive Third,Touches in Midfield Third,Touches in Attacking Third,Touches in Attacking Penalty Area,Dribbles Attempted,Successful Dribbles,Successful Dribble %,Dribbles Tackled,Dribble Tackle %,Total Carries,Total Carry Distance,Progressive Carry Distance,Progressive Carries,Carries into Final Third,Carries into Penalty Area,Carries Miscontrolled,Carries Dispossessed,Total Shots,Shots on Target,Shots on Target %,Goals per Shot,Goals per Shot on Target,Average Shot Distance,Free Kicks Taken,Penalty Kicks Scored,Expected Goals,Non-Penalty Expected Goals,Non-Penalty Expected Goals per Shot,Goal Difference vs Expected Goals,Non-Penalty Goal Difference vs Expected Goals,Passes Completed,Passes Attempted,Pass Completion %,Total Pass Distance,Progressive Pass Distance,Short Passes Completed,Short Passes Attempted,Short Pass Completion %,Medium Passes Completed,Medium Passes Attempted,Medium Pass Completion %,Long Passes Completed,Long Passes Attempted,Long Pass Completion %,Assists,Expected Assists Goals,Expected Assists,Key Passes,Passes into Final Third,Passes into Penalty Area,Crosses into Penalty Area,Progressive Passes,Shots on Target Against,Keeper Saves,Keeper Save Percentage,Clean Sheets,Penalty Kicks Against,Penalty Kicks Against Saved,Penalty Kicks Against Missed,Season,GD,Total_Goals,Points,Points_Cum,GD_Cum,GF_Cum,GA_Cum,Ranking,Minus 1.5 Goals,Minus 2.5 Goals,Minus 3.5 Goals,Points_Cum_Lag,GD_Cum_Lag,GF_Cum_Lag,GA_Cum_Lag,Ranking_Lag,Total Shots_5_Last_Matches_Average,Shots on Target_5_Last_Matches_Average,Shots on Target %_5_Last_Matches_Average,Goals per Shot_5_Last_Matches_Average,Total Touches_5_Last_Matches_Average,Touches in Defensive Penalty Area_5_Last_Matches_Average,Touches in Defensive Third_5_Last_Matches_Average,Touches in Midfield Third_5_Last_Matches_Average,Touches in Attacking Third_5_Last_Matches_Average,Touches in Attacking Penalty Area_5_Last_Matches_Average,Dribbles Attempted_5_Last_Matches_Average,Successful Dribbles_5_Last_Matches_Average,Successful Dribble %_5_Last_Matches_Average,Total Carries_5_Last_Matches_Average,Total Carry Distance_5_Last_Matches_Average,Progressive Carry Distance_5_Last_Matches_Average,Progressive Carries_5_Last_Matches_Average,Carries into Final Third_5_Last_Matches_Average,Carries into Penalty Area_5_Last_Matches_Average,Tackles_5_Last_Matches_Average,Tackles Won_5_Last_Matches_Average,Tackles in Defensive Third_5_Last_Matches_Average,Tackles in Midfield Third_5_Last_Matches_Average,Tackles in Attacking Third_5_Last_Matches_Average,Dribblers Tackled_5_Last_Matches_Average,Total Dribbles Against_5_Last_Matches_Average,Defensive Dribblers Win %_5_Last_Matches_Average,Interceptions_5_Last_Matches_Average,Errors Leading to Goal_5_Last_Matches_Average,Key Passes_5_Last_Matches_Average,Passes Completed_5_Last_Matches_Average,Passes Attempted_5_Last_Matches_Average,Passes into Final Third_5_Last_Matches_Average,Progressive Passes_5_Last_Matches_Average,Shots on Target Against_5_Last_Matches_Average,Keeper Saves_5_Last_Matches_Average,Keeper Save Percentage_5_Last_Matches_Average,Total Shots_5_Last_Matches_Sum,Shots on Target_5_Last_Matches_Sum,Shots on Target %_5_Last_Matches_Sum,Goals per Shot_5_Last_Matches_Sum,Total Touches_5_Last_Matches_Sum,Touches in Defensive Penalty Area_5_Last_Matches_Sum,Touches in Defensive Third_5_Last_Matches_Sum,Touches in Midfield Third_5_Last_Matches_Sum,Touches in Attacking Third_5_Last_Matches_Sum,Touches in Attacking Penalty Area_5_Last_Matches_Sum,Dribbles At

In [209]:
def minmax_scale(x):
    return (x - x.min()) / (x.max() - x.min())

# Ensuite, utilisez cette fonction avec .transform() pour appliquer la normalisation à chaque groupe
RRR = A.groupby(['Season', 'Round'])['Tackles'].transform(minmax_scale)


In [211]:
A.groupby(['Season', 'Round'])['Tackles'].apply(lambda x: x)


Season     Round      
2018-2019  1      8       29.0
                  14      16.0
                  6       15.0
                  7       13.0
                  9       21.0
                          ... 
2023-2024  19     3939    13.0
                  3930    22.0
                  3928    14.0
                  3923    15.0
                  3927    15.0
Name: Tackles, Length: 3940, dtype: float64

In [210]:
RRR

8       1.000000
14      0.350000
6       0.300000
7       0.200000
9       0.600000
          ...   
3939    0.000000
3930    0.500000
3928    0.055556
3923    0.111111
3927    0.111111
Name: Tackles, Length: 3940, dtype: float64

In [148]:
test = Football_viz()
data = test._read_raw_data('Ligue 1')
futur_data = test._read_raw_futur_data('Ligue 1')

In [165]:
processeur = ProcessingFootball()

In [166]:
A = processeur.futur_prediciton_processing(data, futur_data)

KeyError: 'Season'

In [151]:
A

,DateTime,Comp,Round,Day,Venue_Home,Result_Home,GF_Home,GA_Home,Opponent_Home,xG_Home,xGA_Home,Poss_Home,Attendance,Captain_Home,Formation_Home,Referee,Match Report,Notes,Team Home,Season,Minus 1.5 Goals,Minus 2.5 Goals,Minus 3.5 Goals,Points_Cum_Lag_Home,GD_Cum_Lag_Home,GF_Cum_Lag_Home,GA_Cum_Lag_Home,Ranking_Lag_Home,Total Shots_5_Last_Matches_Average_Home,Shots on Target_5_Last_Matches_Average_Home,Shots on Target %_5_Last_Matches_Average_Home,Goals per Shot_5_Last_Matches_Average_Home,Total Touches_5_Last_Matches_Average_Home,Touches in Defensive Penalty Area_5_Last_Matches_Average_Home,Touches in Defensive Third_5_Last_Matches_Average_Home,Touches in Midfield Third_5_Last_Matches_Average_Home,Touches in Attacking Third_5_Last_Matches_Average_Home,Touches in Attacking Penalty Area_5_Last_Matches_Average_Home,Dribbles Attempted_5_Last_Matches_Average_Home,Successful Dribbles_5_Last_Matches_Average_Home,Successful Dribble %_5_Last_Matches_Average_Home,Total Carries_5_Last_Matches_Average_Home,Total Carry Distance_5_Last_Matches_Average_Home,Progressive Carry Distance_5_Last_Matches_Average_Home,Progressive Carries_5_Last_Matches_Average_Home,Carries into Final Third_5_Last_Matches_Average_Home,Carries into Penalty Area_5_Last_Matches_Average_Home,Tackles_5_Last_Matches_Average_Home,Tackles Won_5_Last_Matches_Average_Home,Tackles in Defensive Third_5_Last_Matches_Average_Home,Tackles in Midfield Third_5_Last_Matches_Average_Home,Tackles in Attacking Third_5_Last_Matches_Average_Home,Dribblers Tackled_5_Last_Matches_Average_Home,Total Dribbles Against_5_Last_Matches_Average_Home,Defensive Dribblers Win %_5_Last_Matches_Average_Home,Interceptions_5_Last_Matches_Average_Home,Errors Leading to Goal_5_Last_Matches_Average_Home,Key Passes_5_Last_Matches_Average_Home,Passes Completed_5_Last_Matches_Average_Home,Passes Attempted_5_Last_Matches_Average_Home,Passes into Final Third_5_Last_Matches_Average_Home,Progressive Passes_5_Last_Matches_Average_Home,Shots on Target Against_5_Last_Matches_Average_Home,Keeper Saves_5_Last_Matches_Average_Home,Keeper Save Percentage_5_Last_Matches_Average_Home,Total Shots_5_Last_Matches_Sum_Home,Shots on Target_5_Last_Matches_Sum_Home,Shots on Target %_5_Last_Matches_Sum_Home,Goals per Shot_5_Last_Matches_Sum_Home,Total Touches_5_Last_Matches_Sum_Home,Touches in Defensive Penalty Area_5_Last_Matches_Sum_Home,Touches in Defensive Third_5_Last_Matches_Sum_Home,Touches in Midfield Third_5_Last_Matches_Sum_Home,Touches in Attacking Third_5_Last_Matches_Sum_Home,Touches in Attacking Penalty Area_5_Last_Matches_Sum_Home,Dribbles Attempted_5_Last_Matches_Sum_Home,Successful Dribbles_5_Last_Matches_Sum_Home,Successful Dribble %_5_Last_Matches_Sum_Home,Total Carries_5_Last_Matches_Sum_Home,Total Carry Distance_5_Last_Matches_Sum_Home,Progressive Carry Distance_5_Last_Matches_Sum_Home,Progressive Carries_5_Last_Matches_Sum_Home,Carries into Final Third_5_Last_Matches_Sum_Home,Carries into Penalty Area_5_Last_Matches_Sum_Home,Tackles_5_Last_Matches_Sum_Home,Tackles Won_5_Last_Matches_Sum_Home,Tackles in Defensive Third_5_Last_Matches_Sum_Home,Tackles in Midfield Third_5_Last_Matches_Sum_Home,Tackles in Attacking Third_5_Last_Matches_Sum_Home,Dribblers Tackled_5_Last_Matches_Sum_Home,Total Dribbles Against_5_Last_Matches_Sum_Home,Defensive Dribblers Win %_5_Last_Matches_Sum_Home,Interceptions_5_Last_Matches_Sum_Home,Errors Leading to Goal_5_Last_Matches_Sum_Home,Key Passes_5_Last_Matches_Sum_Home,Passes Completed_5_Last_Matches_Sum_Home,Passes Attempted_5_Last_Matches_Sum_Home,Passes into Final Third_5_Last_Matches_Sum_Home,Progressive Passes_5_Last_Matches_Sum_Home,Shots on Target Against_5_Last_Matches_Sum_Home,Keeper Saves_5_Last_Matches_Sum_Home,Keeper Save Percentage_5_Last_Matches_Sum_Home,Total Shots_5_Last_Matches_Std_Home,Shots on Target_5_Last_Matches_Std_Home,Shots on Target %_5_Last_Matches_Std_Home,Goals per Shot_5_Last_Matches_Std_Home,Total Touches_5_Last_Matches_Std_Home,Touches in Defensive Pen

,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes,Team,Tackles_Tkl,Tackles_TklW,Tackles_Def 3rd,Tackles_Mid 3rd,Tackles_Att 3rd,Challenges_Tkl,Challenges_Att,Challenges_Tkl%,Challenges_Lost,Blocks_Blocks,Blocks_Sh,Blocks_Pass,Int,Tkl+Int,Clr,Err,Touches_Touches,Touches_Def Pen,Touches_Def 3rd,Touches_Mid 3rd,Touches_Att 3rd,Touches_Att Pen,Touches_Live,Take-Ons_Att,Take-Ons_Succ,Take-Ons_Succ%,Take-Ons_Tkld,Take-Ons_Tkld%,Carries_Carries,Carries_TotDist,Carries_PrgDist,Carries_PrgC,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Rec,Receiving_PrgR,Standard_Gls,Standard_Sh,Standard_SoT,Standard_SoT%,Standard_G/Sh,Standard_G/SoT,Standard_Dist,Standard_FK,Standard_PK,Standard_PKatt,Expected_xG,Expected_npxG,Expected_npxG/Sh,Expected_G-xG,Expected_np:G-xG,Total_Cmp,Total_Att,Total_Cmp%,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Short_Cmp%,Medium_Cmp,Medium_Att,Medium_Cmp%,Long_Cmp,Long_Att,Long_Cmp%,Ast,xAG,xA,KP,1/3,PPA,CrsPA,PrgP,Performance_SoTA,Performance_GA,Performance_Saves,Performance_Save%,Performance_CS,Performance_PSxG,Performance_PSxG+/-,Penalty Kicks_PKatt,Penalty Kicks_PKA,Penalty Kicks_PKsv,Penalty Kicks_PKm,Launched_Cmp,Launched_Att,Launched_Cmp%,Passes_Att (GK),Passes_Thr,Passes_Launch%,Passes_AvgLen,Goal Kicks_Att,Goal Kicks_Launch%,Goal Kicks_AvgLen,Crosses_Opp,Crosses_Stp,Crosses_Stp%,Sweeper_#OPA,Sweeper_AvgDist,DateTime
0,Ligue 1,Matchweek 1,Sat,Home,D,0.0,0.0,Lorient,1.2,0.1,78.0,47000.0,Danilo Pereira,4-3-3,Bastien Dechepy,Match Report,NaN,Paris Saint Germain,10.0,7.0,4.0,3.0,3.0,3.0,6.0,50.0,3.0,9.0,1.0,8.0,8.0,18,6.0,0.0,1149.0,24.0,94.0,603.0,461.0,45.0,1149.0,16.0,9.0,56.3,3.0,18.8,1000.0,4664.0,1791.0,29.0,44.0,9.0,15.0,10.0,957.0,62.0,0,20.0,4.0,20.0,0.00,0.00,18.9,1.0,0,0,1.2,1.2,0.06,-1.2,-1.2,964.0,1051.0,91.7,14929.0,3807.0,466.0,484.0,96.3,417.0,441.0,94.6,42.0,62.0,67.7,0.0,1.0,2.2,16.0,134.0,18.0,8.0,62.0,0.0,0.0,0.0,NaN,1.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,NaN,10.0,0.0,0.0,21.5,3.0,0.0,24.0,2.0,0.0,0.0,0.0,NaN,2023-08-12 21:00:00
1,Ligue 1,Matchweek 2,Sat,Away,D,1.0,1.0,Toulouse,2.1,1.2,76.0,27332.0,Marquinhos,4-3-3,Thomas Léonard,Match Report,NaN,Paris Saint Germain,14.0,9.0,2.0,8.0,4.0,9.0,14.0,64.3,5.0,10.0,2.0,8.0,6.0,20,6.0,0.0,886.0,44.0,147.0,553.0,192.0,26.0,885.0,22.0,11.0,50.0,7.0,31.8,756.0,3828.0,1821.0,22.0,20.0,4.0,15.0,7.0,715.0,61.0,1,15.0,2.0,13.3,0.00,0.00,15.9,0.0,1,1,2.1,1.3,0.09,-1.1,-1.3,720.0,802.0,89.8,12251.0,3382.0,326.0,353.0,92.4,322.0,347.0,92.8,53.0,68.0,77.9,0.0,1.2,1.2,13.0,57.0,12.0,2.0,61.0,2.0,1.0,1.0,100.0,0.0,1.0,0.0,1.0,1,0.0,0.0,3.0,3.0,100.0,17.0,3.0,11.8,25.4,6.0,16.7,25.2,6.0,0.0,0.0,0.0,7.0,2023-08-19 21:00:00
2,Ligue 1,Matchweek 3,Sat,Home,W,3.0,1.0,Lens,1.5,0.5,66.0,47864.0,Marquinhos,4-4-2,François Letexier,Match Report,NaN,Paris Saint Germain,15.0,7.0,2.0,10.0,3.0,4.0,10.0,40.0,6.0,10.0,1.0,9.0,4.0,19,11.0,1.0,877.0,102.0,262.0,439.0,183.0,31.0,877.0,36.0,15.0,41.7,18.0,50.0,742.0,3725.0,1485.0,27.0,22.0,10.0,15.0,11.0,690.0,55.0,3,17.0,6.0,35.3,0.18,0.50,19.9,1.0,0,0,1.5,1.5,0.09,1.5,1.5,695.0,761.0,91.3,11830.0,3470.0,290.0,308.0,94.2,343.0,359.0,95.5,45.0,68.0,66.2,3.0,1.4,1.5,15.0,37.0,12.0,1.0,57.0,4.0,1.0,3.0,75.0,0.0,1.1,0.1,0.0,0,0.0,0.0,2.0,8.0,25.0,45.0,6.0,15.6,26.3,8.0,12.5,19.4,16.0,1.0,6.3,0.0,8.8,2023-08-26 21:00:00
3,Ligue 1,Matchweek 4,Sun,Away,W,4.0,1.0,Lyon,4.5,2.1,69.0,51657.0,Marquinhos,4-4-2,Eric Wattellier,Match Report,NaN,Paris Saint Germain,19.0,10.0,7.0,7.0,5.0,14.0,29.0,48.3,15.0,11.0,4.0,7.0,2.0,21,14.0,1.0,961.0,84.0,249.0,513.0,208.0,39.0,960.0,25.0,13.0,52.0,12.0,48.0,812.0,4273.0,1999.0,28.0,23.0,10.0,18.0,11.0,776.0,64.0,4,21.0,7.0,33.3,0.14,0.43,16.0,0.0,1,1,4.5,3.7,0.18,-0.5,-0.7,781.0,843.0,92.6,13098.0,3874.0,364.0,384.0,94.8,332.0,352.0,94.3,59.0,75.0,78.7,2.0,2.7,1.9,18.0,69.0,11.0,0.0,65.0,8.0,1.0,7.0,100.0,0.0,1.9,0.9,1.0,1,0.0,0.0,1.0,4.0,25.0,26.0,3.0,11.5,23.8,8.0,12.5,20.3,13.0,1.0,7.7,0.0,6.5,2023-09-03 20:45:00
4,Ligue 1,Matchwe

In [56]:
futur_data

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes,Team,DateTime
0,2024-01-26,21:00,Ligue 1,Matchweek 19,Fri,Home,NaN,NaN,NaN,Rennes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Lyon,2024-01-26 21:00:00
1,2024-01-26,21:00,Ligue 1,Matchweek 19,Fri,Away,NaN,NaN,NaN,Lyon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Rennes,2024-01-26 21:00:00
2,2024-01-27,17:00,Ligue 1,Matchweek 19,Sat,Away,NaN,NaN,NaN,Nice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Metz,2024-01-27 17:00:00
3,2024-01-27,17:00,Ligue 1,Matchweek 19,Sat,Home,NaN,NaN,NaN,Metz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Nice,2024-01-27 17:00:00
4,2024-01-27,21:00,Ligue 1,Matchweek 19,Sat,Away,NaN,NaN,NaN,Marseille,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Monaco,2024-01-27 21:00:00
5,2024-01-27,21:00,Ligue 1,Matchweek 19,Sat,Home,NaN,NaN,NaN,Monaco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Marseille,2024-01-27 21:00:00
6,2024-01-28,13:00,Ligue 1,Matchweek 19,Sun,Away,NaN,NaN,NaN,Montpellier,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Lille,2024-01-28 13:00:00
7,2024-01-28,13:00,Ligue 1,Matchweek 19,Sun,Home,NaN,NaN,NaN,Lille,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Montpellier,2024-01-28 13:00:00
8,2024-01-28,15:00,Ligue 1,Matchweek 19,Sun,Away,NaN,NaN,NaN,Lorient,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Le Havre,2024-01-28 15:00:00
9,2024-01-28,15:00,Ligue 1,Matchweek 19,Sun,Home,NaN,NaN,NaN,Strasbourg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Clermont Foot,2024-01-28 15:00:00


In [11]:
b = Football_viz()
b.read_data_for_model('Serie A')


,DateTime,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes,Team,Passes Completed,Passes Attempted,Pass Completion %,Total Pass Distance,Progressive Pass Distance,Short Passes Completed,Short Passes Attempted,Short Pass Completion %,Medium Passes Completed,Medium Passes Attempted,Medium Pass Completion %,Long Passes Completed,Long Passes Attempted,Long Pass Completion %,Assists,Expected Assists Goals,Expected Assists,Key Passes,Passes into Final Third,Passes into Penalty Area,Crosses into Penalty Area,Progressive Passes,Tackles,Tackles Won,Tackles in Defensive Third,Tackles in Midfield Third,Tackles in Attacking Third,Dribblers Tackled,Total Dribbles Against,Defensive Dribblers Win %,Defensive Challenges Lost,Total Blocks,Shot Blocks,Pass Blocks,Interceptions,Tackles Plus Interceptions,Clearances,Errors Leading to Goal,Total Shots,Shots on Target,Shots on Target %,Goals per Shot,Goals per Shot on Target,Average Shot Distance,Free Kicks Taken,Penalty Kicks Scored,Expected Goals,Non-Penalty Expected Goals,Non-Penalty Expected Goals per Shot,Goal Difference vs Expected Goals,Non-Penalty Goal Difference vs Expected Goals,Total Touches,Touches in Defensive Penalty Area,Touches in Defensive Third,Touches in Midfield Third,Touches in Attacking Third,Touches in Attacking Penalty Area,Dribbles Attempted,Successful Dribbles,Successful Dribble %,Dribbles Tackled,Dribble Tackle %,Total Carries,Total Carry Distance,Progressive Carry Distance,Progressive Carries,Carries into Final Third,Carries into Penalty Area,Carries Miscontrolled,Carries Dispossessed,Shots on Target Against,Keeper Saves,Keeper Save Percentage,Clean Sheets,Penalty Kicks Against,Penalty Kicks Against Saved,Penalty Kicks Against Missed,Season,GD,Total_Goals,Points,Points_Cum,GD_Cum,GF_Cum,GA_Cum,Ranking,Minus 1.5 Goals,Minus 2.5 Goals,Minus 3.5 Goals,Points_Cum_Lag,GD_Cum_Lag,GF_Cum_Lag,GA_Cum_Lag,Ranking_Lag,Total Shots_5_Last_Matches_Averae,Shots on Target_5_Last_Matches_Averae,Shots on Target %_5_Last_Matches_Averae,Goals per Shot_5_Last_Matches_Averae,Total Touches_5_Last_Matches_Averae,Touches in Defensive Penalty Area_5_Last_Matches_Averae,Touches in Defensive Third_5_Last_Matches_Averae,Touches in Midfield Third_5_Last_Matches_Averae,Touches in Attacking Third_5_Last_Matches_Averae,Touches in Attacking Penalty Area_5_Last_Matches_Averae,Dribbles Attempted_5_Last_Matches_Averae,Successful Dribbles_5_Last_Matches_Averae,Successful Dribble %_5_Last_Matches_Averae,Total Carries_5_Last_Matches_Averae,Total Carry Distance_5_Last_Matches_Averae,Progressive Carry Distance_5_Last_Matches_Averae,Progressive Carries_5_Last_Matches_Averae,Carries into Final Third_5_Last_Matches_Averae,Carries into Penalty Area_5_Last_Matches_Averae,Tackles_5_Last_Matches_Averae,Tackles Won_5_Last_Matches_Averae,Tackles in Defensive Third_5_Last_Matches_Averae,Tackles in Midfield Third_5_Last_Matches_Averae,Tackles in Attacking Third_5_Last_Matches_Averae,Dribblers Tackled_5_Last_Matches_Averae,Total Dribbles Against_5_Last_Matches_Averae,Defensive Dribblers Win %_5_Last_Matches_Averae,Interceptions_5_Last_Matches_Averae,Errors Leading to Goal_5_Last_Matches_Averae,Key Passes_5_Last_Matches_Averae,Passes Completed_5_Last_Matches_Averae,Passes Attempted_5_Last_Matches_Averae,Passes into Final Third_5_Last_Matches_Averae,Progressive Passes_5_Last_Matches_Averae,Shots on Target Against_5_Last_Matches_Averae,Keeper Saves_5_Last_Matches_Averae,Keeper Save Percentage_5_Last_Matches_Averae,Total Shots_5_Last_Matches_Sum,Shots on Target_5_Last_Matches_Sum,Shots on Target %_5_Last_Matches_Sum,Goals per Shot_5_Last_Matches_Sum,Total Touches_5_Last_Matches_Sum,Touches in Defensive Penalty Area_5_Last_Matches_Sum,Touches in Defensive Third_5_Last_Matches_Sum,Touches in Midfield Third_5_Last_Matches_Sum,Touches in Attacking Third_5_Last_Matches_Sum,Touches in Attacking Penalty Area_5_Last_Matches_Sum,Dribbles Attempted_5_Last_Matches_Sum,Successful

In [103]:
b.get_data_for_model('Premier League')

,DateTime,Comp,Round,Day,Venue_Home,Result_Home,GF_Home,GA_Home,Opponent_Home,xG_Home,xGA_Home,Poss_Home,Attendance,Captain_Home,Formation_Home,Referee,Match Report,Notes,Team Home,Season,Minus 1.5 Goals,Minus 2.5 Goals,Minus 3.5 Goals,Points_Cum_Lag_Home,GD_Cum_Lag_Home,GF_Cum_Lag_Home,GA_Cum_Lag_Home,Ranking_Lag_Home,Total Shots_5_Last_Matches_Average_Home,Shots on Target_5_Last_Matches_Average_Home,Shots on Target %_5_Last_Matches_Average_Home,Goals per Shot_5_Last_Matches_Average_Home,Total Touches_5_Last_Matches_Average_Home,Touches in Defensive Penalty Area_5_Last_Matches_Average_Home,Touches in Defensive Third_5_Last_Matches_Average_Home,Touches in Midfield Third_5_Last_Matches_Average_Home,Touches in Attacking Third_5_Last_Matches_Average_Home,Touches in Attacking Penalty Area_5_Last_Matches_Average_Home,Dribbles Attempted_5_Last_Matches_Average_Home,Successful Dribbles_5_Last_Matches_Average_Home,Successful Dribble %_5_Last_Matches_Average_Home,Total Carries_5_Last_Matches_Average_Home,Total Carry Distance_5_Last_Matches_Average_Home,Progressive Carry Distance_5_Last_Matches_Average_Home,Progressive Carries_5_Last_Matches_Average_Home,Carries into Final Third_5_Last_Matches_Average_Home,Carries into Penalty Area_5_Last_Matches_Average_Home,Tackles_5_Last_Matches_Average_Home,Tackles Won_5_Last_Matches_Average_Home,Tackles in Defensive Third_5_Last_Matches_Average_Home,Tackles in Midfield Third_5_Last_Matches_Average_Home,Tackles in Attacking Third_5_Last_Matches_Average_Home,Dribblers Tackled_5_Last_Matches_Average_Home,Total Dribbles Against_5_Last_Matches_Average_Home,Defensive Dribblers Win %_5_Last_Matches_Average_Home,Interceptions_5_Last_Matches_Average_Home,Errors Leading to Goal_5_Last_Matches_Average_Home,Key Passes_5_Last_Matches_Average_Home,Passes Completed_5_Last_Matches_Average_Home,Passes Attempted_5_Last_Matches_Average_Home,Passes into Final Third_5_Last_Matches_Average_Home,Progressive Passes_5_Last_Matches_Average_Home,Shots on Target Against_5_Last_Matches_Average_Home,Keeper Saves_5_Last_Matches_Average_Home,Keeper Save Percentage_5_Last_Matches_Average_Home,Total Shots_5_Last_Matches_Sum_Home,Shots on Target_5_Last_Matches_Sum_Home,Shots on Target %_5_Last_Matches_Sum_Home,Goals per Shot_5_Last_Matches_Sum_Home,Total Touches_5_Last_Matches_Sum_Home,Touches in Defensive Penalty Area_5_Last_Matches_Sum_Home,Touches in Defensive Third_5_Last_Matches_Sum_Home,Touches in Midfield Third_5_Last_Matches_Sum_Home,Touches in Attacking Third_5_Last_Matches_Sum_Home,Touches in Attacking Penalty Area_5_Last_Matches_Sum_Home,Dribbles Attempted_5_Last_Matches_Sum_Home,Successful Dribbles_5_Last_Matches_Sum_Home,Successful Dribble %_5_Last_Matches_Sum_Home,Total Carries_5_Last_Matches_Sum_Home,Total Carry Distance_5_Last_Matches_Sum_Home,Progressive Carry Distance_5_Last_Matches_Sum_Home,Progressive Carries_5_Last_Matches_Sum_Home,Carries into Final Third_5_Last_Matches_Sum_Home,Carries into Penalty Area_5_Last_Matches_Sum_Home,Tackles_5_Last_Matches_Sum_Home,Tackles Won_5_Last_Matches_Sum_Home,Tackles in Defensive Third_5_Last_Matches_Sum_Home,Tackles in Midfield Third_5_Last_Matches_Sum_Home,Tackles in Attacking Third_5_Last_Matches_Sum_Home,Dribblers Tackled_5_Last_Matches_Sum_Home,Total Dribbles Against_5_Last_Matches_Sum_Home,Defensive Dribblers Win %_5_Last_Matches_Sum_Home,Interceptions_5_Last_Matches_Sum_Home,Errors Leading to Goal_5_Last_Matches_Sum_Home,Key Passes_5_Last_Matches_Sum_Home,Passes Completed_5_Last_Matches_Sum_Home,Passes Attempted_5_Last_Matches_Sum_Home,Passes into Final Third_5_Last_Matches_Sum_Home,Progressive Passes_5_Last_Matches_Sum_Home,Shots on Target Against_5_Last_Matches_Sum_Home,Keeper Saves_5_Last_Matches_Sum_Home,Keeper Save Percentage_5_Last_Matches_Sum_Home,Total Shots_5_Last_Matches_Std_Home,Shots on Target_5_Last_Matches_Std_Home,Shots on Target %_5_Last_Matches_Std_Home,Goals per Shot_5_Last_Matches_Std_Home,Total Touches_5_Last_Matches_Std_Home,Touches in Defensive Pen

In [95]:
c

,DateTime,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes,Team,Tackles_Tkl,Tackles_TklW,Tackles_Def 3rd,Tackles_Mid 3rd,Tackles_Att 3rd,Challenges_Tkl,Challenges_Att,Challenges_Tkl%,Challenges_Lost,Blocks_Blocks,Blocks_Sh,Blocks_Pass,Int,Tkl+Int,Clr,Err,Touches_Touches,Touches_Def Pen,Touches_Def 3rd,Touches_Mid 3rd,Touches_Att 3rd,Touches_Att Pen,Touches_Live,Take-Ons_Att,Take-Ons_Succ,Take-Ons_Succ%,Take-Ons_Tkld,Take-Ons_Tkld%,Carries_Carries,Carries_TotDist,Carries_PrgDist,Carries_PrgC,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Rec,Receiving_PrgR,Standard_Gls,Standard_Sh,Standard_SoT,Standard_SoT%,Standard_G/Sh,Standard_G/SoT,Standard_Dist,Standard_FK,Standard_PK,Standard_PKatt,Expected_xG,Expected_npxG,Expected_npxG/Sh,Expected_G-xG,Expected_np:G-xG,Total_Cmp,Total_Att,Total_Cmp%,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Short_Cmp%,Medium_Cmp,Medium_Att,Medium_Cmp%,Long_Cmp,Long_Att,Long_Cmp%,Ast,xAG,xA,KP,1/3,PPA,CrsPA,PrgP,Performance_SoTA,Performance_GA,Performance_Saves,Performance_Save%,Performance_CS,Performance_PSxG,Performance_PSxG+/-,Penalty Kicks_PKatt,Penalty Kicks_PKA,Penalty Kicks_PKsv,Penalty Kicks_PKm,Launched_Cmp,Launched_Att,Launched_Cmp%,Passes_Att (GK),Passes_Thr,Passes_Launch%,Passes_AvgLen,Goal Kicks_Att,Goal Kicks_Launch%,Goal Kicks_AvgLen,Crosses_Opp,Crosses_Stp,Crosses_Stp%,Sweeper_#OPA,Sweeper_AvgDist,Season,GD,Total_Goals,Points
0,2023-08-12 21:00:00,Ligue 1,1,Sat,Home,D,0,0,Lorient,1.2,0.1,78.0,47000.0,Danilo Pereira,4-3-3,Bastien Dechepy,Match Report,NaN,Paris Saint Germain,10.0,7.0,4.0,3.0,3.0,3.0,6.0,50.0,3.0,9.0,1.0,8.0,8.0,18,6.0,0.0,1149.0,24.0,94.0,603.0,461.0,45.0,1149.0,16.0,9.0,56.3,3.0,18.8,1000.0,4664.0,1791.0,29.0,44.0,9.0,15.0,10.0,957.0,62.0,0,20.0,4.0,20.0,0.00,0.00,18.9,1.0,0,0,1.2,1.2,0.06,-1.2,-1.2,964.0,1051.0,91.7,14929.0,3807.0,466.0,484.0,96.3,417.0,441.0,94.6,42.0,62.0,67.7,0.0,1.0,2.2,16.0,134.0,18.0,8.0,62.0,0.0,0.0,0.0,NaN,1.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,NaN,10.0,0.0,0.0,21.5,3.0,0.0,24.0,2.0,0.0,0.0,0.0,NaN,2023-2024,0,0,1
1,2023-08-19 21:00:00,Ligue 1,2,Sat,Away,D,1,1,Toulouse,2.1,1.2,76.0,27332.0,Marquinhos,4-3-3,Thomas Léonard,Match Report,NaN,Paris Saint Germain,14.0,9.0,2.0,8.0,4.0,9.0,14.0,64.3,5.0,10.0,2.0,8.0,6.0,20,6.0,0.0,886.0,44.0,147.0,553.0,192.0,26.0,885.0,22.0,11.0,50.0,7.0,31.8,756.0,3828.0,1821.0,22.0,20.0,4.0,15.0,7.0,715.0,61.0,1,15.0,2.0,13.3,0.00,0.00,15.9,0.0,1,1,2.1,1.3,0.09,-1.1,-1.3,720.0,802.0,89.8,12251.0,3382.0,326.0,353.0,92.4,322.0,347.0,92.8,53.0,68.0,77.9,0.0,1.2,1.2,13.0,57.0,12.0,2.0,61.0,2.0,1.0,1.0,100.0,0.0,1.0,0.0,1.0,1,0.0,0.0,3.0,3.0,100.0,17.0,3.0,11.8,25.4,6.0,16.7,25.2,6.0,0.0,0.0,0.0,7.0,2023-2024,0,2,1
2,2023-08-26 21:00:00,Ligue 1,3,Sat,Home,W,3,1,Lens,1.5,0.5,66.0,47864.0,Marquinhos,4-4-2,François Letexier,Match Report,NaN,Paris Saint Germain,15.0,7.0,2.0,10.0,3.0,4.0,10.0,40.0,6.0,10.0,1.0,9.0,4.0,19,11.0,1.0,877.0,102.0,262.0,439.0,183.0,31.0,877.0,36.0,15.0,41.7,18.0,50.0,742.0,3725.0,1485.0,27.0,22.0,10.0,15.0,11.0,690.0,55.0,3,17.0,6.0,35.3,0.18,0.50,19.9,1.0,0,0,1.5,1.5,0.09,1.5,1.5,695.0,761.0,91.3,11830.0,3470.0,290.0,308.0,94.2,343.0,359.0,95.5,45.0,68.0,66.2,3.0,1.4,1.5,15.0,37.0,12.0,1.0,57.0,4.0,1.0,3.0,75.0,0.0,1.1,0.1,0.0,0,0.0,0.0,2.0,8.0,25.0,45.0,6.0,15.6,26.3,8.0,12.5,19.4,16.0,1.0,6.3,0.0,8.8,2023-2024,2,4,3
3,2023-09-03 20:45:00,Ligue 1,4,Sun,Away,W,4,1,Lyon,4.5,2.1,69.0,51657.0,Marquinhos,4-4-2,Eric Wattellier,Match Report,NaN,Paris Saint Germain,19.0,10.0,7.0,7.0,5.0,14.0,29.0,48.3,15.0,11.0,4.0,7.0,2.0,21,14.0,1.0,961.0,84.0,249.0,513.0,208.0,39.0,960.0,25.0,13.0,52.0,12.0,48.0,812.0,4273.0,1999.0,28.0,23.0,10.0,18.0,11.0,776.0,64.0,4,21.0,7.0,33.3,0.14,0.43,16.0,0.0,1,1,4.5,3.7,0.18,-0.5,-0.7,781.0,843.0,92.6,13098.0,3874.0,364.0,384.0,94.8,332.0,352.0,94.3,59.0,75.0,78.7,2.0,2.7,1.9,18.0,69.0,11.0,0.0,65.0,8.0,1.0,7.0,100.0,0.0,1.9,0.9,1.0,1,0.0,0.0,1.0,4.0,25.0,26.0,3.0,11.5,23.8,8.0,12.5,20.3,13.0,1.0,7.7,0.0,

In [91]:
A = Football_viz()
data = A._read_raw_data('Ligue 1')

In [93]:
B = ProcessingFootball(data, 'Ligue 1')

In [5]:
data= Footbal
a = ProcessingFootball('Ligue 1')

TypeError: ProcessingFootball.__init__() missing 1 required positional argument: 'league'

In [19]:
foot = Football_viz()
a = foot.read_data('Serie A')

AttributeError: 'Football_viz' object has no attribute 'mapped_data'

In [352]:
b = foot.read_data('Eredivisie')

ValueError: cannot convert float NaN to integer

In [350]:
b['Round'].unique()

array(['Matchweek 1', 'Matchweek 2', 'Matchweek 4', 'Matchweek 5',
       'Matchweek 6', 'Matchweek 3', 'Matchweek 7', 'Matchweek 8',
       'Matchweek 9', 'Matchweek 10', 'Matchweek 11', 'Matchweek 12',
       'Matchweek 13', 'Matchweek 14', 'Matchweek 15', 'Matchweek 16',
       'Matchweek 17', 'Matchweek 18', 'Matchweek 19', 'Matchweek 20',
       'Matchweek 21', 'Matchweek 22', 'Matchweek 23', 'Matchweek 24',
       'Matchweek 25', 'Matchweek 26', 'Matchweek 27', 'Matchweek 28',
       'Matchweek 29', 'Matchweek 30', 'Matchweek 31', 'Matchweek 32',
       'Matchweek 33', 'Matchweek 34', 'Semi-finals', 'Finals',
       'Regular season'], dtype=object)

In [351]:
b[b['Round']=='Finals']

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes,Team,Touches_Touches,Touches_Def Pen,Touches_Def 3rd,Touches_Mid 3rd,Touches_Att 3rd,Touches_Att Pen,Touches_Live,Take-Ons_Att,Take-Ons_Succ,Take-Ons_Succ%,Take-Ons_Tkld,Take-Ons_Tkld%,Carries_Carries,Carries_TotDist,Carries_PrgDist,Carries_PrgC,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Rec,Receiving_PrgR,Performance_SoTA,Performance_GA,Performance_Saves,Performance_Save%,Performance_CS,Performance_PSxG,Performance_PSxG+/-,Penalty Kicks_PKatt,Penalty Kicks_PKA,Penalty Kicks_PKsv,Penalty Kicks_PKm,Launched_Cmp,Launched_Att,Launched_Cmp%,Passes_Att (GK),Passes_Thr,Passes_Launch%,Passes_AvgLen,Goal Kicks_Att,Goal Kicks_Launch%,Goal Kicks_AvgLen,Crosses_Opp,Crosses_Stp,Crosses_Stp%,Sweeper_#OPA,Sweeper_AvgDist,Standard_Gls,Standard_Sh,Standard_SoT,Standard_SoT%,Standard_G/Sh,Standard_G/SoT,Standard_Dist,Standard_FK,Standard_PK,Standard_PKatt,Expected_xG,Expected_npxG,Expected_npxG/Sh,Expected_G-xG,Expected_np:G-xG,Total_Cmp,Total_Att,Total_Cmp%,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Short_Cmp%,Medium_Cmp,Medium_Att,Medium_Cmp%,Long_Cmp,Long_Att,Long_Cmp%,Ast,xAG,xA,KP,1/3,PPA,CrsPA,PrgP,Tackles_Tkl,Tackles_TklW,Tackles_Def 3rd,Tackles_Mid 3rd,Tackles_Att 3rd,Challenges_Tkl,Challenges_Att,Challenges_Tkl%,Challenges_Lost,Blocks_Blocks,Blocks_Sh,Blocks_Pass,Int,Tkl+Int,Clr,Err
494,2023-06-08,20:00,Eredivisie,Finals,Thu,Away,D,1,1,Sparta R'dam,1.5,1.3,63.0,10349.0,Robin Pröpper,4-2-3-1,Jeroen Manschot,Match Report,Leg 1 of 2,Twente,671.0,46.0,168.0,283.0,227.0,36.0,671.0,11.0,5.0,45.5,6.0,54.5,443.0,2790.0,1538.0,27.0,20.0,7.0,17.0,7.0,463.0,56.0,2.0,1.0,1.0,100.0,0.0,1.3,0.3,1.0,1.0,0.0,0.0,4.0,11.0,36.4,32.0,8.0,28.1,30.1,7.0,28.6,37.7,10.0,1.0,10.0,3.0,20.8,1,19.0,3.0,15.8,0.05,0.33,18.5,0.0,0,0,1.5,1.5,0.08,-0.5,-0.5,472.0,583.0,81.0,8325.0,2810.0,222.0,240.0,92.5,188.0,222.0,84.7,46.0,84.0,54.8,1,1.0,1.7,12.0,48.0,16.0,4.0,57.0,9.0,7.0,4.0,0.0,5.0,4.0,12.0,33.3,8.0,6.0,3.0,3.0,8.0,17,7.0,0.0
495,2023-06-11,14:30,Eredivisie,Finals,Sun,Home,W,1,0,Sparta R'dam,1.9,0.5,60.0,29500.0,Wout Brama,4-2-3-1,Danny Makkelie,Match Report,Leg 2 of 2; Twente won,Twente,679.0,51.0,121.0,302.0,265.0,46.0,679.0,30.0,15.0,50.0,12.0,40.0,395.0,2194.0,1207.0,29.0,18.0,10.0,13.0,15.0,438.0,51.0,3.0,0.0,3.0,100.0,1.0,0.4,0.4,0.0,0.0,0.0,0.0,2.0,8.0,25.0,20.0,4.0,30.0,34.6,4.0,50.0,38.8,11.0,1.0,9.1,3.0,17.6,1,25.0,9.0,36.0,0.04,0.11,18.3,0.0,0,0,1.9,1.9,0.08,-0.9,-0.9,451.0,564.0,80.0,7932.0,2661.0,213.0,241.0,88.4,186.0,213.0,87.3,43.0,82.0,52.4,1,1.5,2.4,20.0,35.0,15.0,3.0,51.0,8.0,6.0,5.0,1.0,2.0,6.0,16.0,37.5,10.0,7.0,1.0,6.0,5.0,13,17.0,0.0
532,2023-06-08,20:00,Eredivisie,Finals,Thu,Home,D,1,1,Twente,1.3,1.5,37.0,10349.0,Bart Vriends,4-3-3,Jeroen Manschot,Match Report,Leg 1 of 2,Sparta Rotterdam,445.0,82.0,185.0,177.0,88.0,18.0,444.0,17.0,8.0,47.1,4.0,23.5,254.0,1410.0,604.0,11.0,5.0,4.0,14.0,5.0,231.0,17.0,3.0,1.0,2.0,66.7,0.0,0.8,-0.2,0.0,0.0,0.0,0.0,19.0,38.0,50.0,39.0,1.0,66.7,47.4,15.0,80.0,57.5,13.0,1.0,7.7,0.0,11.8,1,8.0,1.0,12.5,0.00,0.00,20.6,0.0,1,1,1.3,0.6,0.07,-0.3,-0.6,233.0,339.0,68.7,4836.0,2435.0,96.0,125.0,76.8,91.0,116.0,78.4,37.0,77.0,48.1,0,0.5,0.7,6.0,21.0,2.0,0.0,17.0,13.0,6.0,8.0,3.0,2.0,6.0,11.0,54.5,5.0,14.0,7.0,7.0,6.0,19,20.0,0.0
533,2023-06-11,14:30,Eredivisie,Finals,Sun,Away,L,0,1,Twente,0.5,1.9,40.0,29500.0,Bart Vriends,4-3-3,Danny Makkelie,Match Report,Leg 2 of 2; Twente won,Sparta Rotterdam,523.0,104.0,257.0,189.0,81.0,17.0,523.0,19.0,10.0,52.6,6.0,31.6,230.0,1505.0,627.0,10.0,11.0,1.0,18.0,2.0,267.0,17.0,9.0,1.0,8.0,88.9,0.0,1.3,0.3,0.0,0.0,0.0,0.0,14.0,24.0,58.3,37.0,5.0,54.1,40.6,9.0,44.4,38.1,20.0,1.0,5.0,1.0,12.5,0,9.0,3.0,33.3,0.00,0.00,15.7,1.0,0,0,0.5,0.5,0.06,-0.5,-0.5,269.0,375.0,71.7,5358.0,2279.0,119.0,148.0,80.4,110.0,134.0,82.1,39.0,79.0,49.4,0,0.4,0.4,6.0,11.0,5.0,1.0,17.0,27.0,16.0,18.0,6.0,3.0,12.0,27.0,44.4,15.0,21.0,11.0,10.0,2.0,29,32.0,0.0
1118,2022-05

In [320]:
a['Date'] = pd.to_datetime(a['Date'])

In [330]:
a[a['Round'] == 'Relegation tie-breaker']

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes,Team,Total_Cmp,Total_Att,Total_Cmp%,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Short_Cmp%,Medium_Cmp,Medium_Att,Medium_Cmp%,Long_Cmp,Long_Att,Long_Cmp%,Ast,xAG,xA,KP,1/3,PPA,CrsPA,PrgP,Tackles_Tkl,Tackles_TklW,Tackles_Def 3rd,Tackles_Mid 3rd,Tackles_Att 3rd,Challenges_Tkl,Challenges_Att,Challenges_Tkl%,Challenges_Lost,Blocks_Blocks,Blocks_Sh,Blocks_Pass,Int,Tkl+Int,Clr,Err,Standard_Gls,Standard_Sh,Standard_SoT,Standard_SoT%,Standard_G/Sh,Standard_G/SoT,Standard_Dist,Standard_FK,Standard_PK,Standard_PKatt,Expected_xG,Expected_npxG,Expected_npxG/Sh,Expected_G-xG,Expected_np:G-xG,Touches_Touches,Touches_Def Pen,Touches_Def 3rd,Touches_Mid 3rd,Touches_Att 3rd,Touches_Att Pen,Touches_Live,Take-Ons_Att,Take-Ons_Succ,Take-Ons_Succ%,Take-Ons_Tkld,Take-Ons_Tkld%,Carries_Carries,Carries_TotDist,Carries_PrgDist,Carries_PrgC,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Rec,Receiving_PrgR,Performance_SoTA,Performance_GA,Performance_Saves,Performance_Save%,Performance_CS,Performance_PSxG,Performance_PSxG+/-,Penalty Kicks_PKatt,Penalty Kicks_PKA,Penalty Kicks_PKsv,Penalty Kicks_PKm,Launched_Cmp,Launched_Att,Launched_Cmp%,Passes_Att (GK),Passes_Thr,Passes_Launch%,Passes_AvgLen,Goal Kicks_Att,Goal Kicks_Launch%,Goal Kicks_AvgLen,Crosses_Opp,Crosses_Stp,Crosses_Stp%,Sweeper_#OPA,Sweeper_AvgDist
1050,2023-06-11,20:45,Serie A,Relegation tie-breaker,Sun,Home,L,1.0,3.0,Hellas Verona,1.9,0.8,68.0,NaN,Dimitris Nikolaou,3-5-2,Daniele Orsato,Match Report,NaN,Spezia,419.0,537.0,78.0,7689.0,2805.0,177.0,199.0,88.9,176.0,213.0,82.6,57.0,103.0,55.3,1.0,0.7,0.8,8.0,35.0,6.0,5.0,36.0,8.0,6.0,4.0,4.0,0.0,5.0,12.0,41.7,7.0,6.0,2.0,4.0,5.0,13.0,19.0,0.0,1.0,16.0,8.0,50.0,0.06,0.13,18.0,1.0,0.0,1.0,1.9,1.4,0.09,-0.9,-0.4,646.0,57.0,211.0,251.0,188.0,25.0,645.0,26.0,9.0,34.6,14.0,53.8,360.0,2042.0,1021.0,24.0,17.0,3.0,20.0,10.0,417.0,36.0,5.0,3.0,2.0,40.0,0.0,0.8,-2.2,0.0,0.0,0.0,0.0,6.0,16.0,37.5,35.0,7.0,40.0,36.4,5.0,40.0,40.0,8.0,1.0,12.5,0.0,10.0
1089,2023-06-11,20:45,Serie A,Relegation tie-breaker,Sun,Away,W,3.0,1.0,Spezia,0.8,1.9,32.0,NaN,Davide Faraoni,3-4-3,Daniele Orsato,Match Report,NaN,Hellas Verona,142.0,250.0,56.8,3324.0,1948.0,54.0,76.0,71.1,52.0,75.0,69.3,33.0,84.0,39.3,2.0,0.6,0.8,8.0,16.0,6.0,0.0,16.0,24.0,16.0,14.0,9.0,1.0,14.0,23.0,60.9,9.0,5.0,1.0,4.0,6.0,30.0,39.0,0.0,3.0,9.0,5.0,55.6,0.33,0.60,16.8,0.0,0.0,0.0,0.8,0.8,0.08,2.2,2.2,373.0,72.0,168.0,125.0,86.0,18.0,373.0,14.0,7.0,50.0,5.0,35.7,130.0,644.0,321.0,6.0,6.0,1.0,14.0,3.0,137.0,16.0,9.0,1.0,7.0,88.9,0.0,1.9,0.9,1.0,0.0,1.0,0.0,14.0,32.0,43.8,23.0,1.0,73.9,53.0,15.0,100.0,71.8,30.0,1.0,3.3,1.0,12.0


In [323]:
a.sort_values(by='Date', ascending=False)

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes,Team,Total_Cmp,Total_Att,Total_Cmp%,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Short_Cmp%,Medium_Cmp,Medium_Att,Medium_Cmp%,Long_Cmp,Long_Att,Long_Cmp%,Ast,xAG,xA,KP,1/3,PPA,CrsPA,PrgP,Tackles_Tkl,Tackles_TklW,Tackles_Def 3rd,Tackles_Mid 3rd,Tackles_Att 3rd,Challenges_Tkl,Challenges_Att,Challenges_Tkl%,Challenges_Lost,Blocks_Blocks,Blocks_Sh,Blocks_Pass,Int,Tkl+Int,Clr,Err,Standard_Gls,Standard_Sh,Standard_SoT,Standard_SoT%,Standard_G/Sh,Standard_G/SoT,Standard_Dist,Standard_FK,Standard_PK,Standard_PKatt,Expected_xG,Expected_npxG,Expected_npxG/Sh,Expected_G-xG,Expected_np:G-xG,Touches_Touches,Touches_Def Pen,Touches_Def 3rd,Touches_Mid 3rd,Touches_Att 3rd,Touches_Att Pen,Touches_Live,Take-Ons_Att,Take-Ons_Succ,Take-Ons_Succ%,Take-Ons_Tkld,Take-Ons_Tkld%,Carries_Carries,Carries_TotDist,Carries_PrgDist,Carries_PrgC,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Rec,Receiving_PrgR,Performance_SoTA,Performance_GA,Performance_Saves,Performance_Save%,Performance_CS,Performance_PSxG,Performance_PSxG+/-,Penalty Kicks_PKatt,Penalty Kicks_PKA,Penalty Kicks_PKsv,Penalty Kicks_PKm,Launched_Cmp,Launched_Att,Launched_Cmp%,Passes_Att (GK),Passes_Thr,Passes_Launch%,Passes_AvgLen,Goal Kicks_Att,Goal Kicks_Launch%,Goal Kicks_AvgLen,Crosses_Opp,Crosses_Stp,Crosses_Stp%,Sweeper_#OPA,Sweeper_AvgDist
4175,2024-01-21,18:00,Serie A,Matchweek 21,Sun,Home,L,1.0,2.0,Genoa,0.8,1.1,60.0,NaN,Antonio Candreva,4-3-2-1,Daniele Orsato,Match Report,NaN,Salernitana,470.0,596.0,78.9,8397.0,2647.0,194.0,220.0,88.2,210.0,247.0,85.0,49.0,84.0,58.3,1.0,0.5,0.7,8.0,31.0,8.0,3.0,37.0,18.0,15.0,8.0,7.0,3.0,7.0,9.0,77.8,2.0,18.0,0.0,18.0,11.0,29.0,13.0,0.0,1.0,11.0,2.0,18.2,0.09,0.50,16.6,1.0,0.0,0.0,0.8,0.8,0.07,0.2,0.2,703.0,51.0,176.0,360.0,172.0,17.0,703.0,17.0,8.0,47.1,8.0,47.1,412.0,1856.0,919.0,12.0,11.0,2.0,12.0,3.0,469.0,37.0,5.0,2.0,3.0,80.0,0.0,1.9,-0.1,1.0,1.0,0.0,0.0,4.0,12.0,33.3,29.0,4.0,34.5,36.2,2.0,100.0,69.0,8.0,1.0,12.5,1.0,23.3
4174,2024-01-21,15:00,Serie A,Matchweek 21,Sun,Home,W,3.0,0.0,Monza,1.3,0.8,28.0,NaN,Sebastiano Luperto,3-5-2,Antonio Giua,Match Report,NaN,Empoli,196.0,284.0,69.0,3981.0,1730.0,73.0,94.0,77.7,103.0,124.0,83.1,20.0,58.0,34.5,1.0,1.2,0.6,6.0,12.0,4.0,0.0,23.0,15.0,8.0,10.0,5.0,0.0,3.0,7.0,42.9,4.0,10.0,5.0,5.0,8.0,23.0,41.0,0.0,3.0,9.0,5.0,55.6,0.33,0.60,16.2,0.0,0.0,0.0,1.3,1.3,0.15,1.7,1.7,388.0,61.0,183.0,151.0,61.0,14.0,388.0,8.0,1.0,12.5,6.0,75.0,195.0,1004.0,455.0,11.0,6.0,2.0,7.0,7.0,195.0,23.0,5.0,0.0,5.0,100.0,1.0,1.4,1.4,0.0,0.0,0.0,0.0,5.0,17.0,29.4,21.0,2.0,57.1,46.3,7.0,71.4,57.9,22.0,1.0,4.5,1.0,17.0
4168,2024-01-21,20:45,Serie A,Matchweek 21,Sun,Away,W,3.0,0.0,Lecce,2.1,0.6,49.0,NaN,Danilo,3-5-2,Daniele Doveri,Match Report,NaN,Juventus,387.0,482.0,80.3,7115.0,2752.0,162.0,187.0,86.6,167.0,191.0,87.4,48.0,80.0,60.0,3.0,2.3,1.2,12.0,36.0,14.0,4.0,41.0,16.0,9.0,7.0,8.0,1.0,2.0,8.0,25.0,6.0,11.0,5.0,6.0,7.0,23.0,21.0,0.0,3.0,15.0,6.0,40.0,0.20,0.50,11.3,0.0,0.0,0.0,2.1,2.1,0.17,0.9,0.9,592.0,68.0,204.0,256.0,140.0,28.0,592.0,13.0,8.0,61.5,5.0,38.5,318.0,1614.0,804.0,16.0,10.0,2.0,16.0,11.0,383.0,41.0,3.0,0.0,3.0,100.0,1.0,0.9,0.9,0.0,0.0,0.0,0.0,6.0,13.0,46.2,27.0,6.0,37.0,33.6,9.0,33.3,31.3,17.0,0.0,0.0,0.0,10.0
4169,2024-01-21,18:00,Serie A,Matchweek 21,Sun,Away,W,2.0,1.0,Salernitana,1.1,0.8,40.0,NaN,Milan Badelj,4-4-2,Daniele Orsato,Match Report,NaN,Genoa,288.0,398.0,72.4,5329.0,2407.0,136.0,168.0,81.0,114.0,130.0,87.7,33.0,67.0,49.3,1.0,0.3,0.4,3.0,24.0,5.0,0.0,30.0,11.0,8.0,7.0,4.0,0.0,8.0,16.0,50.0,8.0,17.0,6.0,11.0,12.0,23.0,31.0,0.0,2.0,4.0,4.0,100.0,0.25,0.25,19.0,0.0,1.0,1.0,1.1,0.4,0.09,0.9,0.6,509.0,66.0,197.0,213.0,103.0,17.0,508.0,9.0,2.0,22.2,7.0,77.8,250.0,1109.0,540.0,11.0,12.0,4.0,8.0,11.0,283.0,28.0,2.0,1.0,1.0,50.0,0.0,1.1,0.1,0.0,0.0,0.0,0.0,11.0,21.0,52.4,32.0,1.0,53.1,41.4,6.0,66.7,45.8,20.0,0.0,0.0,1.0,11.2
4170,2024-01-21,15:00,Serie A,Matchweek 21,Sun,Away